In [71]:
import geopy.distance
import os
from pymongo import MongoClient
from dotenv import load_dotenv

load_dotenv()
MONGO_ADDR = os.environ["MONGO_ADDRESS"]
MONGO_PORT = os.environ["MONGO_PORT"]
DB_NAME = os.environ["MONGO_NOLAN_DB"]
API = os.environ["GOOGLE_API_KEY"]

In [72]:
client = MongoClient(f"mongodb://{MONGO_ADDR}:{MONGO_PORT}/{DB_NAME}")
db = client[DB_NAME]

In [76]:
k = db.users.find_one({"email": "kelvinc@k.com"})
k["password"]

'kelvin'

In [4]:
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.geodesic(coords_1, coords_2).miles)

173.5818455248231


In [74]:
import pandas as pd

In [75]:
df = pd.read_csv("dataset/uszips.csv")

In [85]:
lat_lang_list = list(df[["lat", "lng"]].apply(lambda x: (x[0], x[1]), axis=1).values)
dist = [geopy.distance.geodesic(coords_1, lat_lang).miles for lat_lang in lat_lang_list]

In [1]:
import requests

In [28]:
coords_1

(52.2296756, 21.0122287)

In [91]:
url = f"https://maps.googleapis.com/maps/api/geocode/json?address=la%disneyland&key={API}"

In [92]:
# url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng=37.67831758758746,-122.08425564442709&key={API}'
response = requests.get(url)
data = response.json()

In [96]:
data["results"][0]["address_components"]

[{'long_name': '1313', 'short_name': '1313', 'types': ['street_number']},
 {'long_name': 'Disneyland Drive',
  'short_name': 'Disneyland Dr',
  'types': ['route']},
 {'long_name': 'Anaheim Resort',
  'short_name': 'Anaheim Resort',
  'types': ['neighborhood', 'political']},
 {'long_name': 'Anaheim',
  'short_name': 'Anaheim',
  'types': ['locality', 'political']},
 {'long_name': 'Orange County',
  'short_name': 'Orange County',
  'types': ['administrative_area_level_2', 'political']},
 {'long_name': 'California',
  'short_name': 'CA',
  'types': ['administrative_area_level_1', 'political']},
 {'long_name': 'United States',
  'short_name': 'US',
  'types': ['country', 'political']},
 {'long_name': '92802', 'short_name': '92802', 'types': ['postal_code']}]

In [97]:
def get_zip_lat_lng_from_address(address):
    address = address.replace(" ", "%20")
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={API}'
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        result = data['results'][0]

        # Extract zip code
        zip_code = None
        for component in result['address_components']:
            if 'postal_code' in component['types']:
                zip_code = component['short_name']
                break

        # Extract place type
            place_type = None
            if 'types' in result:
                place_type = result['types'][0]

        # Extract latitude and longitude
        lat = result['geometry']['location']['lat']
        lng = result['geometry']['location']['lng']

        return {
            "zip_code": zip_code,
            "latitude": lat,
            "longitude": lng,
            "place_type": place_type
        }

    return {}

In [98]:
get_zip_lat_lng_from_address("Los Angeles Disneyland")

('92802', 33.8120918, -117.9189742)

In [ ]:
import requests

def get_zip_code_from_lat_lng(latitude, longitude, api_key):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        for result in data['results']:
            for component in result['address_components']:
                if 'postal_code' in component['types']:
                    return component['short_name']
    return None

api_key = 'YOUR_API_KEY'
latitude = 37.4219999
longitude = -122.0840575

zip_code = get_zip_code_from_lat_lng(latitude, longitude, api_key)
print(f'Zip code: {zip_code}')
